# PROJECT:

In [2]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res

In [3]:
df = pd.read_table('lpmc11.dat')
database = db.Database('lpmc', df)
df

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,1,0,0,1,4,3,1,1,1.0,1,...,0.109444,0.000000,0.055556,0.000000,0,0.059444,1.5,0.15,0.0,0.112150
1,13,1,1,1,4,3,1,5,0.0,1,...,0.241389,0.000000,0.122222,0.000000,0,0.132222,0.0,0.50,0.0,0.065126
2,19,4,0,1,3,3,6,5,0.0,1,...,0.222500,0.000000,0.312222,0.000000,0,0.221667,0.0,0.56,0.0,0.086466
3,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.000000,0.062222,0.000000,0,0.117222,0.0,0.41,0.0,0.097156
4,39,9,2,0,4,3,1,5,0.0,1,...,0.146944,0.000000,0.225000,0.000000,0,0.200833,0.0,0.48,0.0,0.378976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,81035,17603,1,1,3,1,1,1,1.0,3,...,0.278056,0.216667,0.000000,0.000000,0,0.392222,2.4,0.98,0.0,0.547450
4996,81040,17604,2,0,3,1,1,2,0.0,3,...,0.264444,0.000000,0.353333,0.176667,1,0.288889,0.0,0.85,0.0,0.175000
4997,81045,17605,0,1,4,3,1,5,0.0,3,...,0.128889,0.000000,0.045833,0.000000,0,0.067778,0.0,0.17,0.0,0.024590
4998,81066,17608,0,3,3,3,6,1,1.0,3,...,0.092222,0.000000,0.389444,0.000000,0,0.193333,1.5,0.61,0.0,0.485632


# Model 0

**Variables:**

In [4]:
dur_walking = Variable('dur_walking') #time for walking
dur_cycling = Variable('dur_cycling') #time for cycling

dur_pt_access = Variable('dur_pt_access')
dur_pt_rail = Variable('dur_pt_rail')
dur_pt_bus = Variable('dur_pt_bus')
dur_pt_int = Variable('dur_pt_int')

dur_pt = dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int #total time for public transport
dur_driving = Variable('dur_driving')

cost_transit = Variable('cost_transit') #cost for public transport
cost_driving_fuel = Variable('cost_driving_fuel')
cost_driving_ccharge = Variable('cost_driving_ccharge')

cost_driving = cost_driving_fuel + cost_driving_ccharge #total cost for driving

travel_mode = Variable('travel_mode') #Choice

**Parameters:**

In [5]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

In [6]:
v_walk_Model0 = (
    constant_walking
    + beta_time * dur_walking
)

v_cycling_Model0 = (
    constant_cycling
    + beta_time * dur_cycling
)

v_PT_Model0 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time * dur_pt
)

v_driving_Model0 = (
    beta_cost * cost_driving
    + beta_time * dur_driving
)

In [7]:
V_Model0 = {1: v_walk_Model0, 2: v_cycling_Model0, 3: v_PT_Model0, 4: v_driving_Model0}
logprob_Model0 = models.loglogit(V_Model0, None, travel_mode)
biogeme_Model0 = bio.BIOGEME(database, logprob_Model0) #create the biogeme estimator
biogeme_Model0.modelName = 'logit_Model0'
res_Model0 = biogeme_Model0.estimate()

In [8]:
print(res_Model0.printGeneralStatistics())

Number of estimated parameters:	5
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4676.875
Final log likelihood:	-4676.875
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00107
Akaike Information Criterion:	9363.751
Bayesian Information Criterion:	9396.337
Final gradient norm:	2.6771E-04
Nbr of threads:	16



In [9]:
res_Model0.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.156585,0.012667,-12.361233,0.0
beta_time,-5.226955,0.191905,-27.237207,0.0
constant_cycling,-2.485329,0.086252,-28.814839,0.0
constant_pt,0.726783,0.047157,15.411971,0.0
constant_walking,1.187012,0.078799,15.063738,0.0


The sign of the 2 generic coefficent leaves us satisfied. We know it's reasonable that utility is decreased when cost and/or travel time increase. It also seems that time impacts more the utility, since in asbolute value beta_time is higher than beta_cost. Concernig the ASCs, we normalized with respect to ASC_car so this is our baseline. We notice that ASC_walking is the highest so it seems that people, assuiming the same atrributes values, would choose this alternative. Anyway we see that, since the value of beta_time is high in absoulute value, and walking usually requires more time than the others, it will be penalized, as we expected. Cycling seems really disadvantaged, since its ASC is already low, and it has the same beta_time (due to generic coefficent assumption). Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

# Model 1

We decide to split beta_time into 4 alternative specific parameters. Our choice is due to the fact that this coefficent is present in every utility function, and we think it can undergo a wider variation, since time perception is strictly related to the chosen travel mode.

In [10]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)

In [11]:
v_walk_Model1 = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_cycling_Model1 = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_PT_Model1 = (
    constant_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model1 = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [12]:
V_Model1 = {1: v_walk_Model1, 2: v_cycling_Model1, 3: v_PT_Model1, 4: v_driving_Model1}
logprob_Model1 = models.loglogit(V_Model1, None, travel_mode)
biogeme_Model1 = bio.BIOGEME(database, logprob_Model1)
biogeme_Model1.modelName = 'logit_Model1'
res_Model1 = biogeme_Model1.estimate()

In [13]:
print(res_Model1.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4372.911
Final log likelihood:	-4372.911
Likelihood ratio test for the init. model:	4.809772e-07
Rho-square for the init. model:	5.5e-11
Rho-square-bar for the init. model:	-0.00183
Akaike Information Criterion:	8761.822
Bayesian Information Criterion:	8813.959
Final gradient norm:	1.9142E-02
Nbr of threads:	16



In [14]:
res_Model1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.136285,0.013922,-9.789189,0.000000e+00
beta_time_cycling,-5.435234,0.492872,-11.027672,0.000000e+00
beta_time_driving,-5.792795,0.384528,-15.064673,0.000000e+00
beta_time_pt,-3.126093,0.243834,-12.820570,0.000000e+00
beta_time_walking,-8.160384,0.386256,-21.126855,0.000000e+00
constant_cycling,-2.560498,0.150687,-16.992113,0.000000e+00
constant_pt,-0.406017,0.069442,-5.846820,5.010597e-09
constant_walking,1.933296,0.129137,14.970913,0.000000e+00


As before, signs are reasonable. Concernign the values of the coefficents, we firstly notice that beta_cost reamined pretty much the same and it seems reasonable becuase people usually care how much money they spend rather than for what they do spend. The wide range of values we can see on beta_time strenghtens our initial assumption, in fact we see how beta_walking is really low, since walking requires phisycal effort, while the two alternatives that just require the individual's focus (driving and cycling) are pretty similar and this seems fine. In the end we see that beta_time_PT is instead the lowest in absolute value, and this is logic for us since when you are in public transport you are able to do other activities or just realx without having to mantain focus. Again, as before, ASC-walking is still bigger than car, as cycling is still the lowest. The only change we notice is the sign of ASC_pt, which is probably due to the value of the beta_time_pt we previously analyzed. Significance of value is confirmed by the t-test values that are all above the 1.96 threshold

In [16]:
res_Model1.likelihood_ratio_test(res_Model0, 0.01)

LRTuple(message='H0 can be rejected at level 1.0%', statistic=607.9289547911903, threshold=11.344866730144373)

To compare the 2 models we used the likelihood ratio test. This is doable since model 0 is a linear restriction of model 1 (assuming all the alternative specific beta_time equal). Our null-hypotesis is that the resricted model (in this case model 0) is the true model. We set the significance level of the test to 0,01 and as we can see from the result the null-hypotesis is rejected, hence our preferred model is model 1

# Model 2:

- Interaction of age with ASCs:

We select 'age' as socio-economic characteristic since we think it has a stornger correlation with the choice of a transport mode rather than sex, which is the other socio-ecomic characteristci we found in the dataset. In fact, if we assume two individuals have the same age, from our experience, their choice (putting all the other parametres equal) would not probably be different. On the other hand, age can impact the choice both from an availability point of view (i.e. young or really old may not have a driving license) and a physical effort one.

In [17]:
age = Variable('age')

In [18]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
ASC_w = Beta('ASC_w', 0, None, None, 0)
ASC_c = Beta('ASC_c', 0, None, None, 0)
ASC_pt = Beta('ASC_pt', 0, None, None, 0)


In [19]:
interacted_ASC_w = ASC_w * age/99
interacted_ASC_c = ASC_c * age/99
interacted_ASC_pt = ASC_pt * age/99


In [20]:
v_w_Model2_ASC = (
    interacted_ASC_w
    + beta_time_walking * dur_walking
)

v_c_Model2_ASC = (
    interacted_ASC_c
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_ASC = (
    interacted_ASC_pt
    + beta_cost * cost_transit
    + beta_time_pt * dur_pt
)

v_driving_Model2_ASC = (
    beta_cost * cost_driving
    + beta_time_driving * dur_driving
)

In [21]:
V_Model2_ASC = {1: v_w_Model2_ASC , 2: v_c_Model2_ASC, 3: v_pt_Model2_ASC, 4: v_driving_Model2_ASC}
logprob_Model2_ASC = models.loglogit(V_Model2_ASC, None, travel_mode)
biogeme_Model2_ASC = bio.BIOGEME(database, logprob_Model2_ASC)
biogeme_Model2_ASC.modelName = 'logit_Model2_ASC'
res_Model2_ASC = biogeme_Model2_ASC.estimate()

In [22]:
print(res_Model2_ASC.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-5359.186
Final log likelihood:	-4615.576
Likelihood ratio test for the init. model:	1487.222
Rho-square for the init. model:	0.139
Rho-square-bar for the init. model:	0.137
Akaike Information Criterion:	9247.151
Bayesian Information Criterion:	9299.289
Final gradient norm:	1.8048E-02
Nbr of threads:	16



In [23]:
res_Model2_ASC.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-4.875802,0.374318,-13.025825,0.000000e+00
ASC_pt,-0.998232,0.133145,-7.497356,6.505907e-14
ASC_w,1.532035,0.158595,9.660054,0.000000e+00
beta_cost,-0.150735,0.014565,-10.348804,0.000000e+00
beta_time_cycling,-8.107216,0.666032,-12.172418,0.000000e+00
beta_time_driving,-5.751766,0.391619,-14.687163,0.000000e+00
beta_time_pt,-3.123984,0.244500,-12.777041,0.000000e+00
beta_time_walking,-5.265422,0.197671,-26.637345,0.000000e+00


Need to find a test! (Probably a Cox test would be ok)

When we have to analyze the interaction of the age with one of the variables, we decided to link it with cost, since we think that indviduals may have a differtent perception of money depending on their age. For example, when you're a student you give a different value to money than the one you give them when you've been working for several years.

- Interaction of age with cost:

In [62]:
constant_walking = Beta('constant_walking', 0, None, None, 0)
constant_cycling = Beta('constant_cycling', 0, None, None, 0)
constant_pt = Beta('constant_pt', 0, None, None, 0)
beta_time_walking = Beta('beta_time_walking', 0, None, None, 0)
beta_time_cycling = Beta('beta_time_cycling', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_driving = Beta('beta_time_driving', 0, None, None, 0)
beta_cost_int = Beta('beta_cost_int', 0, None, None, 0)

In [63]:
beta_cost_interacted = beta_cost_int * age/99

In [64]:
v_w_Model2_cost = (
    constant_walking
    + beta_time_walking * dur_walking
)

v_c_Model2_cost = (
    constant_cycling
    + beta_time_cycling * dur_cycling
)

v_pt_Model2_cost= (
    constant_pt
    + beta_cost_interacted * cost_transit    
    + beta_time_pt * dur_pt
)

v_driving_Model2_cost = (
    beta_cost_interacted * cost_driving
    + beta_time_driving * dur_driving
)

In [65]:
V_Model2_cost = {1: v_w_Model2_cost, 2: v_c_Model2_cost, 3: v_pt_Model2_cost, 4: v_driving_Model2_cost}
logprob_Model2_cost = models.loglogit(V_Model2_cost, None, travel_mode)
biogeme_Model2_cost = bio.BIOGEME(database, logprob_Model2_cost)
biogeme_Model2_cost.modelName = 'logit_Model2_cost'
res_Model2_cost = biogeme_Model2_cost.estimate()

In [66]:
print(res_Model2_cost.printGeneralStatistics())

Number of estimated parameters:	8
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4436.935
Final log likelihood:	-4387.311
Likelihood ratio test for the init. model:	99.24852
Rho-square for the init. model:	0.0112
Rho-square-bar for the init. model:	0.00938
Akaike Information Criterion:	8790.622
Bayesian Information Criterion:	8842.76
Final gradient norm:	1.4147E-02
Nbr of threads:	16



In [67]:
res_Model2_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost_int,-0.287269,0.039547,-7.263920,3.759215e-13
beta_time_cycling,-5.430090,0.494063,-10.990692,0.000000e+00
beta_time_driving,-5.958668,0.382040,-15.596975,0.000000e+00
beta_time_pt,-3.218564,0.242759,-13.258273,0.000000e+00
beta_time_walking,-8.181055,0.386868,-21.146877,0.000000e+00
constant_cycling,-2.566942,0.150921,-17.008496,0.000000e+00
constant_pt,-0.407450,0.069331,-5.876861,4.181197e-09
constant_walking,1.939450,0.129423,14.985404,0.000000e+00


In [68]:
v_w_cox1 = v_walk_Model1 + interacted_ASC_w
v_c_cox1 = v_cycling_Model1 + interacted_ASC_c
v_pt_cox1 = v_PT_Model1 + interacted_ASC_pt
v_driving_cox1 = v_driving_Model1

In [69]:
V_cox1 = {1: v_w_cox1, 2: v_c_cox1, 3: v_pt_cox1, 4: v_driving_cox1}
logprob_cox1 = models.loglogit(V_cox1, None, travel_mode)
biogeme_cox1 = bio.BIOGEME(database, logprob_cox1)
biogeme_cox1.modelName = 'logit_cox1'
res_cox1 = biogeme_cox1.estimate()

In [70]:
print(res_cox1.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4348.347
Final log likelihood:	-4348.347
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00253
Akaike Information Criterion:	8718.694
Bayesian Information Criterion:	8790.383
Final gradient norm:	1.9072E-02
Nbr of threads:	16



In [71]:
res_cox1.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_c,-1.056145,0.365415,-2.890259,3.849241e-03
ASC_pt,-0.959394,0.187749,-5.109975,3.222010e-07
ASC_w,-1.397086,0.236484,-5.907733,3.468465e-09
beta_cost,-0.137214,0.013975,-9.818825,0.000000e+00
beta_time_cycling,-5.424776,0.493259,-10.997821,0.000000e+00
beta_time_driving,-5.741925,0.383465,-14.973784,0.000000e+00
beta_time_pt,-3.112958,0.243471,-12.785768,0.000000e+00
beta_time_walking,-8.186291,0.389650,-21.009325,0.000000e+00
constant_cycling,-2.131860,0.210870,-10.109854,0.000000e+00
constant_pt,-0.012663,0.102425,-0.123633,9.016062e-01


In [72]:
print(res_cox1.likelihood_ratio_test(res_Model1, 0.1))
print(res_cox1.likelihood_ratio_test(res_Model2_ASC, 0.1))

LRTuple(message='H0 can be rejected at level 10.0%', statistic=49.127291194823556, threshold=6.251388631170325)
LRTuple(message='H0 can be rejected at level 10.0%', statistic=534.4566912179689, threshold=6.251388631170325)


In [73]:
v_w_cox2 = v_walk_Model1  
v_c_cox2 = v_cycling_Model1
v_pt_cox2 = v_PT_Model1 + beta_cost_interacted * cost_transit
v_driving_cox2 = v_driving_Model1 + beta_cost_interacted * cost_driving

In [74]:
V_cox2 = {1: v_w_cox2, 2: v_c_cox2, 3: v_pt_cox2, 4: v_driving_cox2}
logprob_cox2 = models.loglogit(V_cox2, None, travel_mode)
biogeme_cox2 = bio.BIOGEME(database, logprob_cox2)
biogeme_cox2.modelName = 'logit_cox2'
res_cox2 = biogeme_cox2.estimate()

In [75]:
print(res_cox2.printGeneralStatistics())

Number of estimated parameters:	9
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4378.043
Final log likelihood:	-4371.267
Likelihood ratio test for the init. model:	13.55326
Rho-square for the init. model:	0.00155
Rho-square-bar for the init. model:	-0.000508
Akaike Information Criterion:	8760.533
Bayesian Information Criterion:	8819.188
Final gradient norm:	2.2540E-02
Nbr of threads:	16



In [77]:
res_cox2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
beta_cost,-0.194856,0.033653,-5.790107,7.034140e-09
beta_cost_int,0.147765,0.079102,1.868034,6.175739e-02
beta_time_cycling,-5.450453,0.493338,-11.048104,0.000000e+00
beta_time_driving,-5.796577,0.384952,-15.057928,0.000000e+00
beta_time_pt,-3.124264,0.244284,-12.789455,0.000000e+00
beta_time_walking,-8.164724,0.386195,-21.141438,0.000000e+00
constant_cycling,-2.559685,0.150777,-16.976647,0.000000e+00
constant_pt,-0.406755,0.069510,-5.851700,4.865730e-09
constant_walking,1.933598,0.129087,14.979019,0.000000e+00


In [78]:
print(res_cox2.likelihood_ratio_test(res_Model1, 0.05))
print(res_cox2.likelihood_ratio_test(res_Model2_cost, 0.05))

LRTuple(message='H0 cannot be rejected at level 5.0%', statistic=3.2885149200665182, threshold=3.841458820694124)
LRTuple(message='H0 can be rejected at level 5.0%', statistic=32.08931322417084, threshold=3.841458820694124)


Preferred model is Model 1